In [ ]:
import math

# Third-party
from astropy.io import ascii
import astropy.coordinates as coord
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as pl
import numpy as np
pl.style.use('apw-notebook')
%matplotlib inline
from scipy import stats

Combining the following quantities:

* total mass of the bulge, $M_b$
* fraction of the bulge that is from destroyed globular clusters, $f$
* mass function of globular clusters, $\frac{d N}{dm}$

I can estimate the initial population of globular clusters (both number and mass distribution). For each cluster, I can get the time of disruption, and therefore the time *since* disruption. Assuming uniform mass-loss, I can estimate the mean densities in position and velocity-space of the resulting streams assuming that the mean fall off is a power law (e.g., Helmi & White 1999). How much substructure is erased if the orbits are strongly chaotic vs. spherical, axisymmetric, triaxial? Can we predict how clumpy the distribution of velocities is in APOGEE and Gaia?

In [ ]:
stats.rv_continuous?

In [ ]:
Mb = 1E9*u.Msun
f = 50E-2 

def dN_dlogm(logm, alpha, beta=2, mc=1E6):
    # power-law with exponential cutoff
    # cutoff mass from ?? (see McLaughlin & Fall 2008)
    # beta = 2 from Zhang & Fall 1999
    m = 10**logm
    return alpha * m**(1-beta) * np.exp(-m/mc)

class MassFn(stats.rv_continuous):
    def _pdf(self,m):
        a = 1007.39
        b = 2
        mc = 1E6
#         if m < 1E3 or m > 1E7:
#             return 0.
#         return a * m**(-b) * np.exp(-m/mc)
        return a / (m*m) * math.exp(-m/mc)
    
massf = MassFn(a=1E3, b=1E7)

In [ ]:
logm = np.linspace(3, 7, 1024)
pl.loglog(10**logm, dN_dlogm(logm, alpha=1., beta=2), marker=None)
pl.xlabel(r"cluster mass [$M_\odot$]")
pl.ylabel(r"$\frac{{\rm d}N}{{\rm d}\,\log{m}}$")
# pl.ylim(0.01,10)

In [ ]:
m_sample = massf.rvs(size=80000)

In [ ]:
fig,axes = pl.subplots(1,2,figsize=(12,6))

bins = np.logspace(3,7,12)

axes[0].hist(m_sample, bins=bins)
axes[0].set_xscale('log')
axes[0].set_yscale('log')

axes[1].hist(m_sample, bins=bins, weights=m_sample)
axes[1].set_xscale('log')
axes[1].set_yscale('log')

In [ ]:
Mb*f / (m_sample.sum()*u.Msun)

In [ ]:
go97 = ascii.read("/Users/adrian/projects/globber/data/gc_catalogs/go97_table1.txt")
go97[:1]

In [ ]:
fig,axes = pl.subplots(1,2,figsize=(12,6))

axes[0].hist(go97['M'], bins=bins)
axes[0].set_xscale('log')
axes[0].set_yscale('log')

axes[1].hist(go97['M'], bins=bins, weights=go97['M'])
axes[1].set_xscale('log')
axes[1].set_yscale('log')